# SQL Query Buddy - Testing Notebook

This notebook demonstrates how to use SQL Query Buddy to convert natural language questions into SQL queries, execute them, and get AI-driven insights.


In [1]:
# Install required packages (run once)
# !pip install -r requirements.txt


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set your OpenAI API key if not in .env file
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Database configuration
DATABASE_URL = os.getenv("DATABASE_URL", "sqlite:///sample_database.db")
VECTOR_DB_PATH = os.getenv("VECTOR_DB_PATH", "./vector_store")


In [3]:
# Setup sample database (run once to create sample data)
from setup_sample_database import create_sample_database

# Create sample database
create_sample_database("sample_database.db")
print("Sample database created!")


Creating tables...
Inserting sample data...
Updating customer statistics...
✅ Sample database created successfully at sample_database.db
   - 100 customers
   - 12 products
   - 500 orders
   - Multiple order items
Sample database created!


In [4]:
# Initialize SQL Query Buddy components
from vector_store import VectorStoreManager
from sql_generator import SQLGenerator
from query_executor import QueryExecutor
from insight_generator import InsightGenerator
from context_manager import ContextManager

# Initialize vector store
print("Initializing vector store...")
vector_store = VectorStoreManager(
    database_url=DATABASE_URL,
    vector_db_path=VECTOR_DB_PATH
)
vector_store.build_vector_store(include_samples=True)
print("✅ Vector store ready!")

# Initialize SQL generator
sql_generator = SQLGenerator(
    vector_store_manager=vector_store,
    model_name="gpt-4-turbo-preview",
    temperature=0.1
)

# Initialize query executor
query_executor = QueryExecutor(
    database_url=DATABASE_URL,
    sql_generator=sql_generator
)

# Initialize insight generator
insight_generator = InsightGenerator(
    model_name="gpt-4-turbo-preview",
    temperature=0.3
)

# Initialize context manager
context_manager = ContextManager(max_history=20)

print("✅ All components initialized!")


Initializing vector store...
Loading database schema...
Loading sample data...
Creating vector store with 9 documents...


/home/unit-s/Documents/Cursor Projects/vector_store.py:109: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  self.vectorstore = Chroma(


Loaded existing vector store.
✅ Vector store ready!
✅ All components initialized!


In [5]:
# Example 1: Simple query
question = "Show me the top 5 customers by total sales"

# Generate SQL
print("Question:", question)
print("\n" + "="*80)
sql_result = sql_generator.generate_sql(
    question=question,
    conversation_history=context_manager.get_conversation_history()
)

print("\nGenerated SQL:")
print(sql_result["sql"])

print("\nExplanation:")
print(sql_result["explanation"])

# Execute query
results = query_executor.execute_safe_query(sql_result["sql"], return_dataframe=True)

if results["success"]:
    print(f"\n✅ Query executed successfully! ({results['row_count']} rows)")
    print("\nResults:")
    display(results["data"])
    
    # Generate insights
    insights = insight_generator.generate_insights(
        query=sql_result["sql"],
        results=results,
        original_question=question
    )
    print("\n" + "="*80)
    print("AI Insights:")
    print(insights)
    
    # Update context
    context_manager.add_exchange(
        question=question,
        sql_query=sql_result["sql"],
        results=results,
        explanation=sql_result["explanation"],
        insights=insights
    )
else:
    print(f"\n❌ Error: {results['error']}")


Question: Show me the top 5 customers by total sales


Generated SQL:
SELECT c.customer_id, c.first_name, c.last_name, SUM(o.total_amount) AS total_sales
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id
ORDER BY total_sales DESC
LIMIT 5

Explanation:
Alright, let's break down this SQL query into simple, easy-to-understand parts. Imagine we're dealing with two lists (or tables, as they're called in SQL): one is a list of customers, and the other is a list of orders. Each customer has an ID, a first name, and a last name. Each order includes information about which customer made the order and how much the order was worth (the total amount).

Here's what the query does, step by step:

1. **Selecting Columns**: The query picks out specific pieces of information to show in the results:
   - The customer's ID (`customer_id`)
   - The customer's first name (`first_name`)
   - The customer's last name (`last_name`)
   - The total amount of money the custom

,customer_id,first_name,last_name,total_sales
0,39,Grace,Jackson,17054.43
1,78,Paul,Moore,13484.36
2,63,Mia,Anderson,13029.43
3,88,Henry,Taylor,12269.49
4,99,Noah,Martin,11399.63



AI Insights:
### Key Findings and Patterns

1. **Top Spender Dominance**: Grace Jackson leads with a total sales amount of $17,054.43, which is significantly higher than the others, indicating a potential VIP customer status.
2. **High Spending Group**: The top 5 customers have a combined total sales of $67,237.34, with each contributing significantly to this total, showcasing a high-value customer segment.

### Notable Statistics

1. **Sales Distribution**: The difference between the highest (Grace Jackson with $17,054.43) and lowest (Noah Martin with $11,399.63) total sales among the top 5 customers is $5,654.80, indicating a wide range of spending within the top tier.
2. **Average Sales**: The average total sales amount among the top 5 customers is $13,447.47, which is quite high, suggesting that these customers are substantially contributing to revenue.

### Anomalies or Outliers

1. **No Middle Ground**: The gap between the highest and the lowest spender among the top 5 is notabl

In [6]:
# Example 2: Follow-up question (uses context from previous query)
question = "Now filter them to California only"

print("Question:", question)
print("\n" + "="*80)

sql_result = sql_generator.generate_sql(
    question=question,
    conversation_history=context_manager.get_conversation_history()
)

print("\nGenerated SQL:")
print(sql_result["sql"])

print("\nExplanation:")
print(sql_result["explanation"])

# Execute query
results = query_executor.execute_safe_query(sql_result["sql"], return_dataframe=True)

if results["success"]:
    print(f"\n✅ Query executed successfully! ({results['row_count']} rows)")
    print("\nResults:")
    display(results["data"])
    
    # Generate insights
    insights = insight_generator.generate_insights(
        query=sql_result["sql"],
        results=results,
        original_question=question
    )
    print("\n" + "="*80)
    print("AI Insights:")
    print(insights)
    
    # Update context
    context_manager.add_exchange(
        question=question,
        sql_query=sql_result["sql"],
        results=results,
        explanation=sql_result["explanation"],
        insights=insights
    )
else:
    print(f"\n❌ Error: {results['error']}")


Question: Now filter them to California only


Generated SQL:
SELECT c.customer_id, c.first_name, c.last_name, SUM(o.total_amount) AS total_sales
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE c.state = 'California'
GROUP BY c.customer_id
ORDER BY total_sales DESC
LIMIT 5

Explanation:
Alright, let's break down this SQL query into simple, easy-to-understand parts. Imagine we're dealing with two lists (or tables) of information. One list is called "customers," and the other is called "orders." Each customer has a unique ID, a first name, and a last name. Each order is linked to a customer (through the customer's unique ID) and has a total amount of money spent on that order.

Here's what the query does, step by step:

1. **Selecting Columns**: We're interested in gathering information about each customer's ID, first name, and last name from the "customers" list. We also want to know the total amount of money each customer has spent on their orders. This total amou

,customer_id,first_name,last_name,total_sales
0,99,Noah,Martin,11399.63
1,2,Grace,Martin,4994.79
2,32,Olivia,Taylor,4849.80
3,80,Yara,Hernandez,4714.59
4,43,Karen,Johnson,2939.46



AI Insights:
Based on the SQL query results for the top 5 customers by total sales in California, here are the key insights and actionable recommendations:

1. **Dominance of Top Customer**: Noah Martin leads with a total sales amount of $11,399.63, which is more than double the sales of the second-highest customer, Grace Martin, at $4,994.79. This indicates a significant reliance on Noah Martin for revenue, suggesting a potential risk if this customer's loyalty is not maintained.

2. **Wide Range in Sales**: The total sales range from $2,939.46 to $11,399.63 among the top 5 customers, showing a wide variance in customer value. This highlights the importance of understanding the different needs and purchasing behaviors of high-value customers to tailor marketing and sales strategies effectively.

3. **Family Ties in Top Spenders**: Notably, two of the top spenders share the last name Martin (Noah and Grace), which could indicate family or business-related purchasing patterns. This mig

## Example 3: Complex Multi-Table Query


In [7]:
# Example 3: Complex query with joins
question = "What's the total revenue from product sales by category this year?"

print("Question:", question)
print("\n" + "="*80)

sql_result = sql_generator.generate_sql(
    question=question,
    conversation_history=context_manager.get_conversation_history()
)

print("\nGenerated SQL:")
print(sql_result["sql"])

print("\nExplanation:")
print(sql_result["explanation"])

# Execute query
results = query_executor.execute_safe_query(sql_result["sql"], return_dataframe=True)

if results["success"]:
    print(f"\n✅ Query executed successfully! ({results['row_count']} rows)")
    print("\nResults:")
    display(results["data"])
    
    # Generate insights
    insights = insight_generator.generate_insights(
        query=sql_result["sql"],
        results=results,
        original_question=question
    )
    print("\n" + "="*80)
    print("AI Insights:")
    print(insights)
    
    # Update context
    context_manager.add_exchange(
        question=question,
        sql_query=sql_result["sql"],
        results=results,
        explanation=sql_result["explanation"],
        insights=insights
    )
else:
    print(f"\n❌ Error: {results['error']}")


Question: What's the total revenue from product sales by category this year?


Generated SQL:
SELECT p.category, SUM(oi.subtotal) AS total_revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.order_date >= '2023-01-01' AND o.order_date <= '2023-12-31'
GROUP BY p.category
ORDER BY total_revenue DESC

Explanation:
Alright, let's break down this SQL query into simple, easy-to-understand parts. Imagine we're dealing with a store's database that keeps track of all the products they sell, the orders made by customers, and the details of what's included in those orders. This query is like asking a very specific question about the store's sales data.

1. **Tables being queried:**
   - The query involves three tables:
     - `order_items`: Contains details about items in each order, like which product was bought and in what quantity.
     - `products`: Holds information about the products, including what category each prod

,category,total_revenue



AI Insights:
No insights available: Query returned no rows.


## Example 4: Query Optimization


In [8]:
# Example 4: Get optimization suggestions
sample_query = """
SELECT c.first_name, c.last_name, o.total_amount, o.order_date
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_date > '2024-01-01'
ORDER BY o.total_amount DESC
"""

print("Original Query:")
print(sample_query)

optimization = query_executor.get_optimization_suggestions(sample_query)

print("\n" + "="*80)
print("Optimization Suggestions:")
print(optimization["suggestions"])


Original Query:

SELECT c.first_name, c.last_name, o.total_amount, o.order_date
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_date > '2024-01-01'
ORDER BY o.total_amount DESC


Optimization Suggestions:
This SQL query retrieves the first name, last name, total amount, and order date of orders from customers that have an order date after January 1, 2024, and orders the results by the total amount in descending order. Here are some optimizations to consider:

1. **Indexes on JOIN and WHERE Columns**:
   - Ensure there is an index on `orders.customer_id` since it's used in a JOIN condition. This will speed up the join operation.
   - An index on `orders.order_date` is crucial because the WHERE clause filters on this column. This will make the date range filter much more efficient.
   - Consider a composite index on `orders(customer_id, order_date)` if most queries filter on `customer_id` and `order_date` together, though in this specific case, since the fil

## Example 5: View Conversation History


In [9]:
# View conversation summary
summary = context_manager.get_summary()
print("Conversation Summary:")
print(f"  Total queries: {summary['total_queries']}")
print(f"  Successful queries: {summary['successful_queries']}")
print(f"  Success rate: {summary['success_rate']:.2%}")
print(f"  Total rows returned: {summary['total_rows_returned']}")
print(f"  Conversation turns: {summary['conversation_turns']}")

print("\n" + "="*80)
print("Recent Context:")
print(context_manager.get_recent_context(n=3))


Conversation Summary:
  Total queries: 3
  Successful queries: 3
  Success rate: 100.00%
  Total rows returned: 10
  Conversation turns: 3

Recent Context:
Recent Conversation Context:

1. Q: Show me the top 5 customers by total sales
   SQL: SELECT c.customer_id, c.first_name, c.last_name, SUM(o.total_amount) AS total_sales
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id
ORDER BY total_sales DESC
LIMIT 5
   Results: 5 rows

2. Q: Now filter them to California only
   SQL: SELECT c.customer_id, c.first_name, c.last_name, SUM(o.total_amount) AS total_sales
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE c.state = 'California'
GROUP BY c.customer_id
ORDER BY total_sales DESC
LIMIT 5
   Results: 5 rows

3. Q: What's the total revenue from product sales by category this year?
   SQL: SELECT p.category, SUM(oi.subtotal) AS total_revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_i

## Example 6: Test RAG Schema Retrieval


In [10]:
# Test RAG: Search for relevant schema information
test_query = "customer sales revenue"

print(f"Searching for relevant schemas for: '{test_query}'")
print("\n" + "="*80)

relevant_schemas = vector_store.search_relevant_schemas(test_query, k=3)

for i, schema in enumerate(relevant_schemas, 1):
    print(f"\n--- Result {i} ---")
    print(schema[:500] + "..." if len(schema) > 500 else schema)


Searching for relevant schemas for: 'customer sales revenue'


--- Result 1 ---
Sample Data:
Sample data from customers:
customer_id | first_name | last_name | email | phone | state | city | registration_date | total_orders | total_spent
--------------------------------------------------------------------------------
1 | Paul | Brown | paul.brown0@email.com | 555-7601 | North Carolina | Philadelphia | 2023-01-29 | 2 | 1014.8500000000001
2 | Zoe | Lee | zoe.lee1@email.com | 555-6445 | North Carolina | Philadelphia | 2023-12-07 | 7 | 8894.55
3 | Bob | Davis | bob.davis2@email...

--- Result 2 ---
Sample Data:
Sample data from orders:
order_id | customer_id | order_date | total_amount | status | shipping_state
--------------------------------------------------------------------------------
1 | 73 | 2024-02-12 | 59.98 | completed | Pennsylvania
2 | 76 | 2024-11-02 | 174.94 | cancelled | Michigan
3 | 69 | 2024-02-01 | 849.9000000000001 | shipped | Florida

--- Result 3 ---
Sample Data:
Samp